# Estimating Credit Transition Matrices with $Cohort$ and $Hazard$ models

In this Python notebook, we use two estimation procedures, the $cohort$ approach and the $hazard$ approach to build historical credit risk transition matrices.

The $cohort$ approach is the traditional technique, much easier to develop. However, it can lead to some transition probabilities being equal to zero! 

For example, as in our available historical data, there are no obligors transiting from the AAA rating class to the CCC rating class, the model attributes to this event, a zero probability of occuring! This might cause a problem, if you are looking to price a particular credit derivative with payoff linked to this event.

In addition, the $cohort$ approach does NOT immidiately allow to calculate the transitions between periods.

For example, if you have estimated the one year transition matrix, you can then easily estimate the two year period by matrix multiplication. However, if you are interested in the probabilities at time 18 months, this is not straithforward to calculate!  

The $hazard$ approach uses the timing and sequencing of transitions within the period (for example a year). 

One consequence is that events so rare in real credit history which are seldom observed empirically, are still given probabilities different from zero. So with this approach you are able to price the above credit derivative.

However, its most important benefit is that, under certain conditions, it allows to create transition probabilities for any period of time.

Please note I have also saved in the folder a very good paper, "Finding Generators for Markov Chains via Empirical Transition Matrices, with Applications to Credit Ratings, by Robert B. Israel, Jeffrey S. Rosenthal and Jason Z. Wei", which explores those conditions. 

What you need when using the cohort approach is the rating at the beginning of the period, and the rating at the end of the period.

The two approaches share the same transition in the numerator. This means that only the final transition in the year is recorded.

However, the two approaches are different in the way the transitions are used in the denominator. The $cohort$ method only uses the rating at the beginning of the period, whereas the $hazard$ method tracks the rating during the year, and weighs the time spent in each rating class accordingly.

We will estimate with the following two ratios, transitions for the Cohort and Hazard model, respectively  



$$p^{Cohort}_{i,j,t}= \frac{N_{i,j,t}}{N_{i,t}} $$


$$p^{Hazard}_{i,j,t}= \frac{N_{i,j,t}}{\int_{t_0}^{t_1} {Y}_i(s)ds} $$

where:
- $N_{i,j,t}$ measures the number of transitions from the rating $i$ to $j$ occuring during time $t$

- $N_{i,t}$ denotes the number of obligors in category $i$ at the beginning of the same period $t$

- $\int_{t_0}^{t_1} {Y}_i(s)ds$ counts the fraction of year each obligor spends in category $i$ between period $t_0$ and $t_1$ 


I have made four main simplifying assumptions when building the two models.
- The goal is to estimate the one year transition matrix, and so our reference period is one year period.
- If the obligor defaults, it can no longer recover and move into another rating category.
- The same happens if the rating category is withdrawn.
- The first rating always runs from the beginning of the year when the rating is given. For example, say the first rating is given on 1st March 2002. We assume that the first rating runs from the 1st January 2002, and until the rating changes. This assumption must be made for the cohort approach. 
For the hazard approach, this is even a stronger assumption, as technicaly we do not need it! We extended it to the hazard model so to keep the way we treat the first "rating" event the same.


The Cohort and Hazard methods are saved in "TransitionClassFile.py" and implemented as a class "TransitionClass".

The class uses several pandas objects to construct the transition events and their times  

In [1]:
import pandas as pd
import numpy  as np
import datetime as dt

In [2]:
from scipy import linalg as la
import TransitionClassFile as TCF

Let's read our data set, from the book:

In [3]:
TransitionData = pd.read_csv ('TransitionHistory.csv', index_col=False)
print (TransitionData.head(2))
print ("---------------------------")
print (TransitionData.tail(2))
size = len(TransitionData)

   Id       Date RatingSymbol
0   1  30-May-00          CCC
1   1  31-Dec-00            B
---------------------------
        Id       Date RatingSymbol
3925  1829  30-Dec-99           AA
3926  1829  30-Aug-00           NR


In [4]:
print (TransitionData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3927 entries, 0 to 3926
Data columns (total 3 columns):
Id              3927 non-null int64
Date            3927 non-null object
RatingSymbol    3927 non-null object
dtypes: int64(1), object(2)
memory usage: 92.1+ KB
None


In [5]:
print (TransitionData.dtypes)

Id               int64
Date            object
RatingSymbol    object
dtype: object


In [6]:
MappingData = pd.read_csv ('RatingMapping.csv', index_col=False)
print (MappingData)

  RatingSymbol  RatingNumber
0          AAA             0
1           AA             1
2            A             2
3          BBB             3
4           BB             4
5            B             5
6          CCC             6
7            D             7
8           NR             8


In [7]:
TransitionData = pd.merge(TransitionData, MappingData, on='RatingSymbol', how='left')
TransitionData.head()

,Id,Date,RatingSymbol,RatingNumber
0,1,30-May-00,CCC,6
1,1,31-Dec-00,B,5
2,2,21-May-03,B,5
3,3,30-Dec-99,BB,4
4,3,30-Oct-00,B,5


In [8]:
TransitionData.Date = pd.to_datetime(TransitionData.Date)

In [9]:
print (TransitionData.dtypes)

Id                       int64
Date            datetime64[ns]
RatingSymbol            object
RatingNumber             int64
dtype: object


In [10]:
ystart = min(TransitionData.Date)
ystart = ystart.year
ystart

1999

In [11]:
yend = max(TransitionData.Date)
yend = yend.year
yend

2005

In [12]:
classes = max(TransitionData.RatingNumber)
classes

8

In [13]:
size = len(TransitionData)
size

3927

In [14]:
obs = len(TransitionData)
obs

3927

In [15]:
TransitionData['Date'] = TransitionData['Date'].dt.date

### Let's practise with the TransitionClass

In [16]:
# Use the Id number 7 and 11 to get used to the outputs
# for the obligor with Id 7
ObligorID = 7 # type 11 next time
MyDati = TransitionData[TransitionData['Id'] == ObligorID] 
MyDati = MyDati.copy()
MyDati

,Id,Date,RatingSymbol,RatingNumber
13,7,2002-12-30,BBB,3
14,7,2003-06-23,BB,4
15,7,2003-12-30,B,5
16,7,2004-05-21,BB,4


In [17]:
example1 = TCF.TransitionClass(MyDati, yend)
example1.Cohort()
example1.CohortTransitionMatrix()
example1.HazardModel()

In [18]:
# We print the ratings symbols at the beginning and at the end of each year respectively 
print(example1.RatingsBeg)
print(example1.RatingsEnd)
      

['BBB', 'BBB', 'B', 'BB']
['BBB', 'B', 'BB', 'BB']


In [19]:
# We print the denominator array, which counts the number of positions at the beginning of each year
# The position 1 corresponds to AAA, position 2 to AA, and the last one is Defaut
print(example1.TransDen)

[0. 0. 0. 2. 1. 1. 0. 0.]


In [20]:
# The sum of TransDen is the total of years under investigation for this obligor
print(sum(example1.TransDen))

4.0


In [21]:
# We print the transition matrix, which counts the number of transits from one rating (row) to another rating (column)
print(example1.TransMatrix)

[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [22]:
# TransDenLambda is the denominator for the hazard method. 
print("Hazard: ", example1.TransDenLambda)
print("----------------")
print("Cohort: ", example1.TransDen)

Hazard:  [0.         0.         0.         1.4739726  2.13424658 0.39178082
 0.         0.        ]
----------------
Cohort:  [0. 0. 0. 2. 1. 1. 0. 0.]


In [23]:
# The sum of TransDenLambda should again be equal to the sum of TransDen.
# Not exactly the same, as the time spent in each rating class is divided by 365. 
# To be excact we should have used the exact number of days in each year (2000 and 2004 are leap years)
print(sum(example1.TransDenLambda))
print(sum(example1.TransDen))

4.0
4.0


### Let's run the TransitionClass() on all data available!

In [24]:
maxID = TransitionData.iloc[size-1].Id

In [25]:
# The arrays containing the outputs
TransMatrixResults       = np.zeros([8, 9])
TransDenResults          = np.zeros([8])
TransDenLambdaResults    = np.zeros([8])

In [26]:
# Run the algo for all obligors
for i in range(1, maxID + 1):
            
    MyDati = TransitionData[TransitionData['Id'] == i] 
    print ("processing ID borrower No: ", i)

    if (len(MyDati) == 1 and MyDati.iloc[0].RatingSymbol == 'NR'):
        # when this is true the data is not processed as it contains only one record, equal to state NR 
        print("Only one NR Rating event on Borrower No: ", i)
    
    else:
        trans = TCF.TransitionClass(MyDati, yend)
        trans.Cohort()
        trans.CohortTransitionMatrix()
        trans.HazardModel()

        TransMatrixResults    = trans.TransMatrix + TransMatrixResults  
        TransDenResults       = trans.TransDen    + TransDenResults 
        TransDenLambdaResults = trans.TransDenLambda + TransDenLambdaResults


processing ID borrower No:  1
processing ID borrower No:  2
processing ID borrower No:  3
processing ID borrower No:  4
processing ID borrower No:  5
processing ID borrower No:  6
processing ID borrower No:  7
processing ID borrower No:  8
processing ID borrower No:  9
processing ID borrower No:  10
processing ID borrower No:  11
processing ID borrower No:  12
processing ID borrower No:  13
processing ID borrower No:  14
processing ID borrower No:  15
processing ID borrower No:  16
processing ID borrower No:  17
processing ID borrower No:  18
processing ID borrower No:  19
Only one NR Rating event on Borrower No:  19
processing ID borrower No:  20
processing ID borrower No:  21
processing ID borrower No:  22
processing ID borrower No:  23
processing ID borrower No:  24
Only one NR Rating event on Borrower No:  24
processing ID borrower No:  25
processing ID borrower No:  26
processing ID borrower No:  27
processing ID borrower No:  28
processing ID borrower No:  29
processing ID borrow

processing ID borrower No:  229
processing ID borrower No:  230
processing ID borrower No:  231
processing ID borrower No:  232
processing ID borrower No:  233
processing ID borrower No:  234
processing ID borrower No:  235
processing ID borrower No:  236
Only one NR Rating event on Borrower No:  236
processing ID borrower No:  237
processing ID borrower No:  238
processing ID borrower No:  239
processing ID borrower No:  240
processing ID borrower No:  241
processing ID borrower No:  242
processing ID borrower No:  243
processing ID borrower No:  244
processing ID borrower No:  245
processing ID borrower No:  246
processing ID borrower No:  247
processing ID borrower No:  248
processing ID borrower No:  249
processing ID borrower No:  250
processing ID borrower No:  251
processing ID borrower No:  252
processing ID borrower No:  253
processing ID borrower No:  254
processing ID borrower No:  255
processing ID borrower No:  256
processing ID borrower No:  257
processing ID borrower No:

processing ID borrower No:  458
processing ID borrower No:  459
processing ID borrower No:  460
processing ID borrower No:  461
Only one NR Rating event on Borrower No:  461
processing ID borrower No:  462
processing ID borrower No:  463
Only one NR Rating event on Borrower No:  463
processing ID borrower No:  464
processing ID borrower No:  465
processing ID borrower No:  466
Only one NR Rating event on Borrower No:  466
processing ID borrower No:  467
processing ID borrower No:  468
processing ID borrower No:  469
processing ID borrower No:  470
processing ID borrower No:  471
processing ID borrower No:  472
processing ID borrower No:  473
processing ID borrower No:  474
processing ID borrower No:  475
processing ID borrower No:  476
processing ID borrower No:  477
processing ID borrower No:  478
processing ID borrower No:  479
processing ID borrower No:  480
processing ID borrower No:  481
processing ID borrower No:  482
processing ID borrower No:  483
processing ID borrower No:  48

processing ID borrower No:  713
processing ID borrower No:  714
processing ID borrower No:  715
processing ID borrower No:  716
processing ID borrower No:  717
processing ID borrower No:  718
processing ID borrower No:  719
processing ID borrower No:  720
processing ID borrower No:  721
processing ID borrower No:  722
processing ID borrower No:  723
Only one NR Rating event on Borrower No:  723
processing ID borrower No:  724
processing ID borrower No:  725
Only one NR Rating event on Borrower No:  725
processing ID borrower No:  726
processing ID borrower No:  727
processing ID borrower No:  728
processing ID borrower No:  729
processing ID borrower No:  730
processing ID borrower No:  731
Only one NR Rating event on Borrower No:  731
processing ID borrower No:  732
processing ID borrower No:  733
processing ID borrower No:  734
processing ID borrower No:  735
processing ID borrower No:  736
processing ID borrower No:  737
processing ID borrower No:  738
processing ID borrower No:  73

processing ID borrower No:  939
processing ID borrower No:  940
processing ID borrower No:  941
processing ID borrower No:  942
processing ID borrower No:  943
processing ID borrower No:  944
processing ID borrower No:  945
processing ID borrower No:  946
processing ID borrower No:  947
processing ID borrower No:  948
processing ID borrower No:  949
Only one NR Rating event on Borrower No:  949
processing ID borrower No:  950
Only one NR Rating event on Borrower No:  950
processing ID borrower No:  951
processing ID borrower No:  952
processing ID borrower No:  953
processing ID borrower No:  954
processing ID borrower No:  955
processing ID borrower No:  956
processing ID borrower No:  957
processing ID borrower No:  958
processing ID borrower No:  959
processing ID borrower No:  960
processing ID borrower No:  961
processing ID borrower No:  962
processing ID borrower No:  963
processing ID borrower No:  964
processing ID borrower No:  965
Only one NR Rating event on Borrower No:  96

processing ID borrower No:  1170
processing ID borrower No:  1171
processing ID borrower No:  1172
processing ID borrower No:  1173
processing ID borrower No:  1174
processing ID borrower No:  1175
Only one NR Rating event on Borrower No:  1175
processing ID borrower No:  1176
processing ID borrower No:  1177
processing ID borrower No:  1178
processing ID borrower No:  1179
Only one NR Rating event on Borrower No:  1179
processing ID borrower No:  1180
Only one NR Rating event on Borrower No:  1180
processing ID borrower No:  1181
processing ID borrower No:  1182
processing ID borrower No:  1183
processing ID borrower No:  1184
Only one NR Rating event on Borrower No:  1184
processing ID borrower No:  1185
processing ID borrower No:  1186
processing ID borrower No:  1187
processing ID borrower No:  1188
processing ID borrower No:  1189
processing ID borrower No:  1190
processing ID borrower No:  1191
processing ID borrower No:  1192
processing ID borrower No:  1193
processing ID borrow

processing ID borrower No:  1391
processing ID borrower No:  1392
processing ID borrower No:  1393
processing ID borrower No:  1394
processing ID borrower No:  1395
processing ID borrower No:  1396
processing ID borrower No:  1397
processing ID borrower No:  1398
processing ID borrower No:  1399
processing ID borrower No:  1400
processing ID borrower No:  1401
processing ID borrower No:  1402
processing ID borrower No:  1403
processing ID borrower No:  1404
processing ID borrower No:  1405
processing ID borrower No:  1406
processing ID borrower No:  1407
processing ID borrower No:  1408
processing ID borrower No:  1409
processing ID borrower No:  1410
processing ID borrower No:  1411
processing ID borrower No:  1412
processing ID borrower No:  1413
processing ID borrower No:  1414
processing ID borrower No:  1415
processing ID borrower No:  1416
processing ID borrower No:  1417
processing ID borrower No:  1418
processing ID borrower No:  1419
processing ID borrower No:  1420
processing

processing ID borrower No:  1610
processing ID borrower No:  1611
Only one NR Rating event on Borrower No:  1611
processing ID borrower No:  1612
processing ID borrower No:  1613
processing ID borrower No:  1614
processing ID borrower No:  1615
processing ID borrower No:  1616
processing ID borrower No:  1617
processing ID borrower No:  1618
processing ID borrower No:  1619
processing ID borrower No:  1620
processing ID borrower No:  1621
processing ID borrower No:  1622
processing ID borrower No:  1623
processing ID borrower No:  1624
processing ID borrower No:  1625
processing ID borrower No:  1626
processing ID borrower No:  1627
Only one NR Rating event on Borrower No:  1627
processing ID borrower No:  1628
processing ID borrower No:  1629
processing ID borrower No:  1630
processing ID borrower No:  1631
processing ID borrower No:  1632
processing ID borrower No:  1633
processing ID borrower No:  1634
processing ID borrower No:  1635
processing ID borrower No:  1636
processing ID b

In [27]:
TransDenResults

array([ 158., 1195., 2336., 2098.,  970.,  772.,  242.,   11.])

In [28]:
TransDenLambdaResults

array([ 157.30958904, 1170.37260274, 2303.56712329, 2062.93972603,
        950.75890411,  775.23835616,  252.57260274,   60.01643836])

In [29]:
# In case you need to see the transitions 
TransMatrixResults
pij = pd.DataFrame(TransMatrixResults)
pij.to_csv('pij.csv')

In [30]:
RatingMgrationCohort = np.zeros([8, 9])
RatingMgrationHazard = np.zeros([8, 9])
for i in range(8):
    for j in range(9):
        RatingMgrationCohort[i,j] = TransMatrixResults[i,j] / TransDenResults[i]
        RatingMgrationHazard[i,j] = TransMatrixResults[i,j] / TransDenLambdaResults[i]

#Default Category
RatingMgrationHazard[7,7] = 1 # Absorbing
RatingMgrationHazard[7,8] = 0 # All the others = 0

### The Generator Matrix for the Hazard Approach

In [31]:
for i in range(8):
    RatingMgrationHazard[i,i] = 0
    RatingMgrationHazard[i,i] = -sum(RatingMgrationHazard[i,:])

In [32]:
RatingMgrationHazardDF = pd.DataFrame(RatingMgrationHazard)
# Change the column names 
RatingMgrationHazardDF.columns =['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR'] 
  
# Change the row indexes 
RatingMgrationHazardDF.index = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default']  

In [33]:
# printing the data frame 
RatingMgrationHazardDF

,AAA,AA,A,BBB,BB,B,CCC,Default,NR
AAA,-0.063569,0.012714,0.000000,0.000000,0.006357,0.000000,0.000000,0.000000,0.044498
AA,0.011108,-0.102531,0.058956,0.001709,0.000000,0.000000,0.000000,0.000000,0.030759
A,0.000868,0.021705,-0.098543,0.042109,0.002171,0.001302,0.000000,0.000000,0.030388
BBB,0.000000,0.000000,0.029085,-0.111491,0.043627,0.010664,0.001939,0.001454,0.024722
BB,0.000000,0.000000,0.004207,0.064159,-0.224032,0.090454,0.011570,0.006311,0.047331
B,0.000000,0.001290,0.002580,0.005160,0.065786,-0.187039,0.056757,0.010319,0.045147
CCC,0.000000,0.000000,0.000000,0.003959,0.011878,0.059389,-0.296944,0.083144,0.138574
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000


In [34]:
# Let's check. All Rows should add to ZERO!
RatingMgrationHazardDF.sum(axis=1)

AAA        0.000000e+00
AA         3.469447e-18
A         -6.938894e-18
BBB        3.469447e-18
BB        -2.775558e-17
B         -1.387779e-17
CCC       -2.775558e-17
Default    0.000000e+00
dtype: float64

### Cohort Approach Transition Matrix 

In [35]:
# Let's prepare the Cohort Matrix
# add a zero row to the cohort matrix
row_to_be_added = np.zeros((9))
RatingMgrationCohort_V2 = np.vstack ((RatingMgrationCohort, row_to_be_added) )
RatingMgrationCohort_V2[8,8] = 1.0

In [36]:
RatingMgrationCohortDF = pd.DataFrame(RatingMgrationCohort_V2)
# Change the column names 
RatingMgrationCohortDF.columns =['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR'] 
  
# Change the row indexes 
RatingMgrationCohortDF.index = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR']  
  
# printing the data frame 
RatingMgrationCohortDF

,AAA,AA,A,BBB,BB,B,CCC,Default,NR
AAA,0.936709,0.012658,0.000000,0.000000,0.006329,0.000000,0.000000,0.000000,0.044304
AA,0.010879,0.899582,0.057741,0.001674,0.000000,0.000000,0.000000,0.000000,0.030126
A,0.000856,0.021404,0.902825,0.041524,0.002140,0.001284,0.000000,0.000000,0.029966
BBB,0.000000,0.000000,0.028599,0.890372,0.042898,0.010486,0.001907,0.001430,0.024309
BB,0.000000,0.000000,0.004124,0.062887,0.780412,0.088660,0.011340,0.006186,0.046392
B,0.000000,0.001295,0.002591,0.005181,0.066062,0.812176,0.056995,0.010363,0.045337
CCC,0.000000,0.000000,0.000000,0.004132,0.012397,0.061983,0.690083,0.086777,0.144628
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
NR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [37]:
# All rows should add to ONE!
RatingMgrationCohortDF.sum(axis=1)

AAA        1.0
AA         1.0
A          1.0
BBB        1.0
BB         1.0
B          1.0
CCC        1.0
Default    1.0
NR         1.0
dtype: float64

### The Transition Matrix for the Hazard Approach 

To be able to compare the Hazard vs the Cohort approach, we need to prepare the transition matrix from a generator.

To do so we have to run an operator called $Matrix$ $ Exponential$ of a generator matrix!

We have used in the past two methods to do so:

A first one, which is a very convoluted one, in our view, which we hope we have correctly implemented, and

a second where we use the eigenvalues and eigenvectors of the generator!

In [38]:
#First add a zero row to the generator matrix
# Array to be added as row
row_to_be_added = np.zeros((9))
 # Adding row to numpy array
RatingMgrationHazard_V2 = np.vstack ((RatingMgrationHazard, row_to_be_added) )

#### Method 1

In [39]:
#1 Calculate the maximum negative number of the generator, lmax
#2 Create a diagonal matrix with lmax as main diagonal 
#3 Add the diagonal matrix in 2 to the generator to calcuate lStar
#4 Calculate the matrix exponential of (Lstar)
#5 Mutiply Exp(-1 * lmax) by the matrix in 4

In [40]:
#1
lmax = 0
for i in range(8):
    if (np.abs(RatingMgrationHazard[i, i]) > lmax):
        lmax = np.abs(RatingMgrationHazard[i, i])
print (lmax)

0.2969443209059649


In [41]:
#2
mat1 = np.zeros((9, 9))
np.fill_diagonal(mat1, lmax)

In [42]:
#3
Lstar = RatingMgrationHazard_V2 +  mat1

In [43]:
#4
tmp = la.expm(Lstar)

In [44]:
vec1 = np.zeros((9, 9))
np.fill_diagonal(vec1, np.exp(-lmax))

In [45]:
mexpgenerator = np.dot(vec1, tmp)

In [46]:
mexpgeneratorDF = pd.DataFrame(mexpgenerator)
# Change the column names 
mexpgeneratorDF.columns =['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR'] 
  
# Change the row indexes 
mexpgeneratorDF.index = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR']  

#### Hazard Approach Transition Matrix 

In [47]:
# printing the data frame 
mexpgeneratorDF.round(6)

,AAA,AA,A,BBB,BB,B,CCC,Default,NR
AAA,0.938475,0.011704,0.000357,0.000194,0.005520,0.000247,0.000035,0.000020,0.043447
AA,0.010249,0.903193,0.053363,0.002658,0.000136,0.000050,0.000003,0.000002,0.030347
A,0.000911,0.019644,0.907297,0.038026,0.002691,0.001437,0.000084,0.000046,0.029864
BBB,0.000013,0.000292,0.026302,0.896296,0.037305,0.010937,0.002070,0.001642,0.025142
BB,0.000002,0.000096,0.004500,0.054667,0.802946,0.074368,0.011023,0.006581,0.045817
B,0.000008,0.001144,0.002477,0.006380,0.054043,0.833249,0.044959,0.011635,0.046105
CCC,0.000000,0.000033,0.000140,0.003708,0.010792,0.047168,0.744456,0.072278,0.121425
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
NR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [48]:
# and compare it with the cohort matrix
RatingMgrationCohortDF.round(6)

,AAA,AA,A,BBB,BB,B,CCC,Default,NR
AAA,0.936709,0.012658,0.000000,0.000000,0.006329,0.000000,0.000000,0.000000,0.044304
AA,0.010879,0.899582,0.057741,0.001674,0.000000,0.000000,0.000000,0.000000,0.030126
A,0.000856,0.021404,0.902825,0.041524,0.002140,0.001284,0.000000,0.000000,0.029966
BBB,0.000000,0.000000,0.028599,0.890372,0.042898,0.010486,0.001907,0.001430,0.024309
BB,0.000000,0.000000,0.004124,0.062887,0.780412,0.088660,0.011340,0.006186,0.046392
B,0.000000,0.001295,0.002591,0.005181,0.066062,0.812176,0.056995,0.010363,0.045337
CCC,0.000000,0.000000,0.000000,0.004132,0.012397,0.061983,0.690083,0.086777,0.144628
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
NR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


Before moving to method 2, a quick comment on the one year trasition matrices as prepared with the Hazard and cohort methods.

The values in the main diagonal are very similar. We can only notice that CCC rating has a higher probability of default according to the cohort method.

Very importantly instead, notice the many zero entries in the cohort matrix, meaning impossible events to occur!

Conversely, the hazard method is more relax and does not create any impossible events. The only 0 value, from CCC to AAA is due to rounding! 

#### Method 2 - Eigenvalues and Eigenvectors of the Generator Matrix

The idea is that you are able to reconstruct the generator by using its eigenvalues and eigenvectors

In [49]:
evals, evecs = la.eig(RatingMgrationHazard_V2)

In [50]:
size = len(evals)
arr1 = np.zeros((size, size))
np.fill_diagonal(arr1, np.real(evals))

In [51]:
Recunstructed = np.dot(np.dot(evecs,arr1),np.linalg.inv(evecs))

In [52]:
Recunstructed = pd.DataFrame(Recunstructed)
Recunstructed.columns =['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR'] 
Recunstructed.index = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR']  

In [53]:
Recunstructed.round(6)

,AAA,AA,A,BBB,BB,B,CCC,Default,NR
AAA,-0.063569,0.012714,0.000000,0.000000,0.006357,-0.000000,-0.000000,0.000000,0.044498
AA,0.011108,-0.102531,0.058956,0.001709,0.000000,0.000000,0.000000,-0.000000,0.030759
A,0.000868,0.021705,-0.098543,0.042109,0.002171,0.001302,-0.000000,0.000000,0.030388
BBB,-0.000000,0.000000,0.029085,-0.111491,0.043627,0.010664,0.001939,0.001454,0.024722
BB,-0.000000,0.000000,0.004207,0.064159,-0.224032,0.090454,0.011570,0.006311,0.047331
B,-0.000000,0.001290,0.002580,0.005160,0.065786,-0.187039,0.056757,0.010319,0.045147
CCC,0.000000,0.000000,-0.000000,0.003959,0.011878,0.059389,-0.296944,0.083144,0.138574
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
NR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [54]:
# and it is just the same original matrix 
RatingMgrationHazardDF

,AAA,AA,A,BBB,BB,B,CCC,Default,NR
AAA,-0.063569,0.012714,0.000000,0.000000,0.006357,0.000000,0.000000,0.000000,0.044498
AA,0.011108,-0.102531,0.058956,0.001709,0.000000,0.000000,0.000000,0.000000,0.030759
A,0.000868,0.021705,-0.098543,0.042109,0.002171,0.001302,0.000000,0.000000,0.030388
BBB,0.000000,0.000000,0.029085,-0.111491,0.043627,0.010664,0.001939,0.001454,0.024722
BB,0.000000,0.000000,0.004207,0.064159,-0.224032,0.090454,0.011570,0.006311,0.047331
B,0.000000,0.001290,0.002580,0.005160,0.065786,-0.187039,0.056757,0.010319,0.045147
CCC,0.000000,0.000000,0.000000,0.003959,0.011878,0.059389,-0.296944,0.083144,0.138574
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000


Now we exponentiate the eigenvalues and reconstruct the Transition matrix

In [55]:
arr1 = np.zeros((size, size))
np.fill_diagonal(arr1, np.real(np.exp(evals)))
Recunstructed = np.dot(np.dot(evecs,arr1),np.linalg.inv(evecs))
Recunstructed = pd.DataFrame(Recunstructed)
Recunstructed.columns =['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR'] 
Recunstructed.index = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR']  
Recunstructed.round(6)

,AAA,AA,A,BBB,BB,B,CCC,Default,NR
AAA,0.938475,0.011704,0.000357,0.000194,0.005520,0.000247,0.000035,0.000020,0.043447
AA,0.010249,0.903193,0.053363,0.002658,0.000136,0.000050,0.000003,0.000002,0.030347
A,0.000911,0.019644,0.907297,0.038026,0.002691,0.001437,0.000084,0.000046,0.029864
BBB,0.000013,0.000292,0.026302,0.896296,0.037305,0.010937,0.002070,0.001642,0.025142
BB,0.000002,0.000096,0.004500,0.054667,0.802946,0.074368,0.011023,0.006581,0.045817
B,0.000008,0.001144,0.002477,0.006380,0.054043,0.833249,0.044959,0.011635,0.046105
CCC,0.000000,0.000033,0.000140,0.003708,0.010792,0.047168,0.744456,0.072278,0.121425
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
NR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [56]:
# and it is just the same as calculated with method 1
mexpgeneratorDF.round(6)

,AAA,AA,A,BBB,BB,B,CCC,Default,NR
AAA,0.938475,0.011704,0.000357,0.000194,0.005520,0.000247,0.000035,0.000020,0.043447
AA,0.010249,0.903193,0.053363,0.002658,0.000136,0.000050,0.000003,0.000002,0.030347
A,0.000911,0.019644,0.907297,0.038026,0.002691,0.001437,0.000084,0.000046,0.029864
BBB,0.000013,0.000292,0.026302,0.896296,0.037305,0.010937,0.002070,0.001642,0.025142
BB,0.000002,0.000096,0.004500,0.054667,0.802946,0.074368,0.011023,0.006581,0.045817
B,0.000008,0.001144,0.002477,0.006380,0.054043,0.833249,0.044959,0.011635,0.046105
CCC,0.000000,0.000033,0.000140,0.003708,0.010792,0.047168,0.744456,0.072278,0.121425
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
NR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


### Multi period Transitions

There is one main advantage when using an Hazard Transition Matrix:

We can calculate transition matrix for any period we want, say 6, 9, 15 or 18 months. 

**There are however some conditions for this to be true. See the research paper from Israel, Rosenthal and Wei 

We KNOW!! It is very EASY to prepare a transition matrix over a two year period!

we only need to run a matrix multiplication.

The two transition matrices, based on the cohort and hazard aproach, are as follows.

In [57]:
RatingMgrationCohort_V2_year2 = np.dot(RatingMgrationCohort_V2, RatingMgrationCohort_V2)

RatingMgrationCohortDF_year2 = pd.DataFrame(RatingMgrationCohort_V2_year2)
RatingMgrationCohortDF_year2.columns =['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR'] 
RatingMgrationCohortDF_year2.index = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR']  

mexpgenerator_year2 = np.dot(mexpgenerator, mexpgenerator)

mexpgenerator_year2DF_year2 = pd.DataFrame(mexpgenerator_year2)
mexpgenerator_year2DF_year2.columns =['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR'] 
mexpgenerator_year2DF_year2.index = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR']  

#### 2 year Transition Matrix - Hazard Approach

In [58]:
mexpgenerator_year2DF_year2

,AAA,AA,A,BBB,BB,B,CCC,Default,NR
AAA,0.880856,0.021563,0.001314,0.000704,0.009636,0.000853,0.000131,0.000081,0.084861
AA,0.018924,0.816927,0.096687,0.006821,0.000534,0.000205,0.000020,0.000013,0.059870
A,0.001884,0.035589,0.825252,0.068792,0.006108,0.003122,0.000312,0.000191,0.058752
BBB,0.000050,0.001059,0.047649,0.806465,0.064075,0.021826,0.004302,0.003637,0.050937
BB,0.000010,0.000353,0.009325,0.093579,0.650912,0.122805,0.020514,0.013616,0.088884
B,0.000028,0.002044,0.004790,0.014253,0.089155,0.700518,0.071541,0.024945,0.092727
CCC,0.000001,0.000113,0.000496,0.006980,0.019388,0.075261,0.556462,0.126711,0.214588
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
NR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


#### 2 year Transition Matrix - Cohort Approach

In [59]:
RatingMgrationCohortDF_year2 

,AAA,AA,A,BBB,BB,B,CCC,Default,NR
AAA,0.877561,0.023244,0.000757,0.000419,0.010868,0.000561,0.000072,0.000039,0.086479
AA,0.020026,0.810621,0.104120,0.005393,0.000264,0.000092,0.000003,0.000002,0.059479
A,0.001808,0.038591,0.817529,0.074638,0.005474,0.002828,0.000177,0.000086,0.058869
BBB,0.000024,0.000626,0.051487,0.796709,0.072451,0.021811,0.004097,0.003243,0.049551
BB,0.000004,0.000203,0.008969,0.105747,0.617748,0.142566,0.021849,0.013006,0.089908
B,0.000016,0.002273,0.004938,0.013321,0.106144,0.669078,0.086380,0.024141,0.093709
CCC,0.000000,0.000080,0.000330,0.007632,0.022501,0.094258,0.479895,0.147385,0.247919
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
NR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


### However, with the generator, as estimated with the hazard approach, we are no longer restricted to integer years!!! 

In [60]:
# Let's prepare a simple function to do just this!
def FutureTransition(Matrix, time):
    # Time is the future date. It can be 2 (for 2 years), or 1.5 (for 18 months) 
    # Matrix is the Generator matrix, M by M matrix, a square matrix!
    
    evals, evecs = la.eig(Matrix) # it calculate eigenvalues and eigenvectors   
    size = len(evals)
    arr1 = np.zeros((size, size))
    
    np.fill_diagonal(arr1, np.exp(time*np.real(evals)))
    
    FutMatrix = np.dot(np.dot(evecs,arr1),np.linalg.inv(evecs))
    FutMatrix = pd.DataFrame(FutMatrix)
    FutMatrix.columns = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR'] 
    FutMatrix.index   = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'Default', 'NR']  
    
    return FutMatrix.round(6)

In [61]:
# so using the generator over ONE year, we should recover the ONE YEAR transition Matrix.
# and INDEED!!
FutureTransition(RatingMgrationHazard_V2,1)

,AAA,AA,A,BBB,BB,B,CCC,Default,NR
AAA,0.938475,0.011704,0.000357,0.000194,0.005520,0.000247,0.000035,0.000020,0.043447
AA,0.010249,0.903193,0.053363,0.002658,0.000136,0.000050,0.000003,0.000002,0.030347
A,0.000911,0.019644,0.907297,0.038026,0.002691,0.001437,0.000084,0.000046,0.029864
BBB,0.000013,0.000292,0.026302,0.896296,0.037305,0.010937,0.002070,0.001642,0.025142
BB,0.000002,0.000096,0.004500,0.054667,0.802946,0.074368,0.011023,0.006581,0.045817
B,0.000008,0.001144,0.002477,0.006380,0.054043,0.833249,0.044959,0.011635,0.046105
CCC,0.000000,0.000033,0.000140,0.003708,0.010792,0.047168,0.744456,0.072278,0.121425
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
NR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [62]:
# Over 2 years
FutureTransition(RatingMgrationHazard_V2,2)

,AAA,AA,A,BBB,BB,B,CCC,Default,NR
AAA,0.880856,0.021563,0.001314,0.000704,0.009636,0.000853,0.000131,0.000081,0.084861
AA,0.018924,0.816927,0.096687,0.006821,0.000534,0.000205,0.000020,0.000013,0.059870
A,0.001884,0.035589,0.825252,0.068792,0.006108,0.003122,0.000312,0.000191,0.058752
BBB,0.000050,0.001059,0.047649,0.806465,0.064075,0.021826,0.004302,0.003637,0.050937
BB,0.000010,0.000353,0.009325,0.093579,0.650912,0.122805,0.020514,0.013616,0.088884
B,0.000028,0.002044,0.004790,0.014253,0.089155,0.700518,0.071541,0.024945,0.092727
CCC,0.000001,0.000113,0.000496,0.006980,0.019388,0.075261,0.556462,0.126711,0.214588
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
NR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [63]:
# and over 18 months
FutureTransition(RatingMgrationHazard_V2,1.5)

,AAA,AA,A,BBB,BB,B,CCC,Default,NR
AAA,0.909194,0.016849,0.000771,0.000416,0.007731,0.000517,0.000076,0.000046,0.064402
AA,0.014770,0.858823,0.076178,0.004585,0.000303,0.000114,0.000010,0.000006,0.045211
A,0.001392,0.028041,0.865032,0.054237,0.004335,0.002250,0.000182,0.000106,0.044424
BBB,0.000028,0.000625,0.037542,0.849794,0.051829,0.016434,0.003175,0.002597,0.037974
BB,0.000005,0.000207,0.006894,0.075821,0.722053,0.101309,0.015989,0.010050,0.067672
B,0.000016,0.001621,0.003651,0.010206,0.073586,0.763181,0.060120,0.018157,0.069462
CCC,0.000001,0.000068,0.000297,0.005393,0.015362,0.063160,0.643283,0.101401,0.171036
Default,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
NR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


As a final remark, you may ask, why not to estract the eigenvalues and eigenvectors of the cohort transition matrix.

You may try and be successfull. You will be on this data! 

However, as the numbers of transitions with probabilities equal to zero increases, it becomes more unlikely that you end up with a set of real eigenvalues and eigenvectors as and being able to $decompose$ the matrix. 

FINITO!!!!